# Building the Question Answering System

In [1]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from keras.models import Model, load_model
from keras.layers import Input, Dense, GRU, Masking, Lambda, Bidirectional, Dropout, Reshape
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
import keras.backend as K
import tensorflow as tf

from EpisodicMemoryModule import EpisodicMemoryModule

Using TensorFlow backend.


# Preprocessing

### Reading in Training and Dev Datasets
It should be noted that the most recent version of the Stanford Question Answering dataset (SQuAD 2.0) includes 'impossible questions' as denoted in the dataset's '*is_impossible*' column that was not part of the dataset in earlier versions. The reason for this (according to the SQuAD website) is to encourage research into Question Answering systems that can cope with un-answerable questions by not even attempting to answer them (by returning empty string outputs or else). Given the added complexity this brings to the project, you will see in the code below that I simply exclude 'impossible questions' in this project.

In [2]:
train_df_all=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/training_data.json").reset_index(drop=True)
dev_df_all=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/validation_data.json").reset_index(drop=True)

# Dataframes consisting only of 'possible' to answer questions.
train_df = train_df_all[train_df_all.is_impossible==False][["context", "question", "answer_text",
                                                    "answer_start", "title"]].reset_index(drop=True)

dev_df = dev_df_all[dev_df_all.is_impossible==False][["context", "question", "answer_text",
                                                    "answer_start", "title"]].reset_index(drop=True)

# Simple Processing/Adding answer_end
train_df.answer_start = train_df.answer_start.astype(int)
dev_df.answer_start = dev_df.answer_start.astype(int)

contexts, questions, answers, answer_start = (train_df.context.values,
                                              train_df.question.values, 
                                              train_df.answer_text.values,
                                              train_df.answer_start.values)
answer_end = np.array([answer_start[idx] + len(answers[idx]) for idx in range(len(answer_start))])

## Word Vectorization
To be able to use words, phrases, questions or other natural language constructs in our model we require a to provide our neural network a numerical representation of our words (as these are the elemental NLP 'particles'). The simplest implementation would be to use 'one hot encoding' and define each word as a vector the size of our dictionary (the number of unique words found in our collection of documents, our corpus). However, this approach will most likely be insufficient for the purposes of a question answering system. word2vec and GloVe are 2 popular choices sophisticated options for word embeddings that also capture word similarities. I will not go into the details of either architecture other than to say that we will not be re-training the word vectors due to the insufficient size of the dataset, and we will begin with the GloVe word embeddings due to it's superior performance in most 'downstream' modelling tasks. Having said that, given the simplicity of swapping word vector representations we will also test out performance with word2vec (providing we can do so in a time-efficient manner).

Info and download links for GloVe can be found at: https://nlp.stanford.edu/projects/glove/

### Word Vector Custom Function

In [3]:
def get_word_vector_dict(url_or_path):
    """ Input: url_or_path, a URL or a local path.
        Output: word_vector_dict, a dictionary of GloVe word vectors where words are the keys
        and the values are the word vectors with the dimension specified in the input file."""
    with open(url_or_path, encoding="utf8") as glove_text:
        word_embeddings = [line.split(" ") for line in glove_text.readlines()]
    word_vector_dict = {element[0]:list(map(float, element[1:])) for element in word_embeddings}
    
    return word_vector_dict

### Setting up the Word Vectors


In [4]:
word_vector_50_dict = get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.50d.txt")
vocab = np.array(word_vector_50_dict.keys()) #400k words as per the documentation.
word_vector_100_dict = get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.100d.txt")

### Model Hyperparameters

In [5]:
word_vector_size = 50
hidden_units = 50
input_h_units = int(hidden_units/2)
num_memory_passes = 3
regularization_val = 1e-4
dropout = 0.1

### Tokenization, Embedding, Padding Custom Functions

In [6]:
def tokenize(string):
    """Takes a string (either context, question or answer) and returns the string as a list of tokens.
    Input: string, a str object.
    Output: a list of tokens, where each token is a substring."""
    tokens = [token.replace("``", '"').replace("''", '"').lower() for token in word_tokenize(string)]
    split_tokens = []
    
    for token in tokens:
        split_tokens.extend(re.split('(\W+)', token)) # split further
    
    return [token for token in split_tokens if token!=" " and token!=""]


def get_embedding(tokens):
    """Takes a tokenized context, question or answer and returns its vectorized/embedded form.
    Input: tokens, list of tokens of a string.
    Output: embedding, a numpy array of the vectorized/embedded representation of the string."""
    assert word_vector_size==50 or word_vector_size==100
    tokens = np.array(tokens)
    embedding=[]
    
    if word_vector_size==50:
        word_vector_dict=word_vector_50_dict
    else:
        word_vector_dict=word_vector_100_dict
    
    for token in tokens:
        if token in word_vector_dict.keys(): 
            embedding.extend(word_vector_dict[token])
        else:
            # Words with no embedding are assigned the 'unk' token vectorization (already in GloVe)
            embedding.extend(word_vector_dict["unk"])
    
    return np.array(embedding)


def get_sent_end_idx(context_tokenizations):
    """ Get indices of tokens that are '.' (sentence end tokens). For one or many contexts.
    Input: context_tokenizations, a list or numpy array of 1 or more tokenized contexts.
    Output: a numpy array of indices of sentence end tokens."""
    return np.array([np.where(np.array(context)==".")[0] for context in context_tokenizations])


def get_padded_inputs(tokenized_inputs, string_type="context"):
    """ Takes a numpy array of tokenized inputs and returns embedding and padding.
    Input: input_array, a numpy array of question or answer tokenizations.
    Output: (embedded_input, padded_input) a tuple of numpy arrays."""
    assert isinstance(tokenized_inputs[0], list)==True # Assert multiple samples
    embedded_inputs = [get_embedding(tokenized_input) for tokenized_input in tokenized_inputs]
    
    if string_type=="context":
        padded_input = pad_sequences(embedded_inputs, max_context_len*word_vector_size, padding="post",
                                dtype=np.float16).reshape(len(tokenized_inputs), -1, word_vector_size)
    elif string_type=="question":
        padded_input = pad_sequences(embedded_inputs, max_question_len*word_vector_size, padding="post",
                                dtype=np.float16).reshape(len(tokenized_inputs), -1, word_vector_size)
    else:
        print("Incorrect string_type parameter value.")
    
    return padded_input


def get_answer_span(answer_start, answer_end):
    """ Returns one hot numpy matrices for the answer_start and answer_end indices.
       Input: answer_start, numpy array containing the 'answer start' index in the context
              answer_end, numpy array containing the 'answer end' index in the context
       Output: tuple of size 2, containing the one hot embeddings of the indices for each context"""
    y_answer_start, y_answer_end= ([] , [])
    start_arr, end_arr = (np.zeros(shape=(output_dim,), dtype=float),
                          np.zeros(shape=(output_dim,), dtype=float)) # Set the 0.0 arrays
  
    if isinstance(answer_start, np.int32) and isinstance(answer_end, np.int32): # Single sample case
        start_arr[answer_start]=1.0
        end_arr[answer_end]=1.0
        y_answer_start, y_answer_end = start_arr, end_arr 
   
    else:
        assert len(answer_start)==len(answer_end)
        for sample_idx in range(len(answer_start)):  # Multi sample case
            start_arr[answer_start[sample_idx]]=1.0
            end_arr[answer_end[sample_idx]]=1.0
        
            y_answer_start.append(start_arr)
            y_answer_end.append(end_arr)
        
            start_arr, end_arr = (np.zeros(shape=(output_dim,), dtype=float),
                          np.zeros(shape=(output_dim,), dtype=float)) # Reset the 0.0 arrays
        
    return (np.array(y_answer_start), np.array(y_answer_end))   

The nltk tokenizer generated around 110,00 unique tokens from our contexts, questions and answers in our dataset. ~31,000 of those tokens did not have pre-trained word vectorizations in the GloVe model. Some of these tokens were numbers, expressed as strings in an unfamiliar format, some of these tokens were misspelled words, some of these tokens were works in other languages, or symbols from other alphibets and so on. 
With the 'regex inspired' split in the tokenized function, we were able to reduce the number of words with no embeddings to around 16,000. To deal with the remaining words with no embeddings we assigned to them the embedding for the token *'unk'*, which by definition is the embedding for unknown words provided by GloVe. Thus any word/token that did not have an embedding got an *'unk'* embedding.

### Setting up 'Trans-Batch' Variables
All the computations needed for each batch training step will be computed later, the variables defined below have to take on values now as they are required in downstream calculations.

In [7]:
""" Calculating the 'trans-batch' variables. EXECUTE THESE BEFORE ANY CHANGES TO ANYTHING"""
tokenized_contexts = np.array([tokenize(context) for context in contexts])
tokenized_questions = np.array([tokenize(question) for question in questions])
tokenized_answers = np.array([tokenize(answer) for answer in answers])
sent_end_indices = get_sent_end_idx(tokenized_contexts)

# Calculating variables used within the model architecture
max_context_len = np.max([len(context) for context in tokenized_contexts])
max_question_len = np.max([len(question) for question in tokenized_questions])
max_answer_len = np.max([len(answer) for answer in tokenized_answers])
max_sent_num = np.max([len(idxs) for idxs in sent_end_indices])

output_dim = np.max([len(context) for context in contexts]) 

# Building the Dynamic Memory Network
### Training Hyperparameters

In [8]:
num_of_samples = len(list(train_df.context.values))
num_of_epochs = 1
batch_size = 128
num_of_batches = int(num_of_samples/batch_size) # Number of iterations
learning_rate = 0.001

#num_of_batches = int(num_of_samples/batch_size) + 1 ONLY USE IF batch_size DOES NOT DIVIDE num_of_samples

## Defining Custom Functions

In [9]:
def get_facts(facts_output, sent_end_idxs):
    """ Extracts the timesteps (facts) for each sample, then pads each tensor. Returns a 3D tensor.
        Input: facts_output, 3D tensor of all the timesteps/samples. Shape=(num samples, max # words, 50)
        Output: padded_fact_tensors, 3D tensor of padded fact embedding representations. """      
    def extract_facts(input_tuple):
        timesteps, padded_indices = input_tuple 
        
        num_facts = tf.count_nonzero(input_tensor=padded_indices, dtype=tf.int32, keepdims=True) 
        indices = tf.slice(input_=padded_indices, begin=tf.zeros([1,], tf.int32), size=num_facts)
 
        facts_tensor = tf.nn.embedding_lookup(timesteps, ids=indices)
        
        pad = tf.zeros(shape=[max_sent_num-tf.shape(facts_tensor)[0], hidden_units], dtype=tf.float32)
        padded_facts_tensor = K.concatenate([facts_tensor, pad], axis=0)
        
        return padded_facts_tensor
    
    input_tuple = (facts_output, sent_end_idxs)
    padded_fact_tensors = tf.map_fn(fn=extract_facts, elems=input_tuple, dtype=tf.float32)  
    
    return padded_fact_tensors

## Input Module

- **What it does:** The Input Module iterates over each context (paragraph of a wikipedia article) and returns embedded representations (facts) of each sentence in the context (for each sample).
- **How it does this:** The Input Module uses a Bidirectional GRU that iterates over each word and returns the hidden state after each iteration. The GRU requires the input to be a 3D Tensor of shape (samples, timesteps, columns/features), and each sample must have the same (timesteps, columns/features) shape. However, given that our contexts are of different length (different amounts of words/timesteps), the embedded representations are padded (have 0.0s appended to each context embedding) as to meet the input requirements. As a technical side note, all embedded contexts are padded such that their length is equal to the length of the longest context in the whole sample (again, such that each individual sample has the same shape). To ensure that the GRU interprets the 0.0s as paddings, we mask the input (via the Masking layer). A Dropout Layer is added as a form of regularization. The *get_facts* method then returns the facts for each sample/context by extracting the hidden states/outputs of the GRU corresponding to 'end of sentence token' timesteps (exactly as described in the *'Ask Me Anything...'* paper.

In [10]:
context_input = Input(shape=(max_context_len, word_vector_size), name="ContextInput")

sent_end_idx_input = Input(shape=(max_sent_num, ), dtype=tf.int32, name="IndicesInput") 

context_mask = Masking(mask_value=0.0, name="ContextMask")(context_input)

facts_output = Bidirectional(GRU(units=input_h_units, return_sequences=True, dropout=dropout,
                        kernel_regularizer=regularizers.l2(regularization_val),
                        recurrent_regularizer=regularizers.l2(regularization_val)),
                        merge_mode="concat", name="ContextBiGRU")(context_mask)

facts_tensors = Lambda(get_facts, arguments={"sent_end_idxs":sent_end_idx_input},
                                           name="FactTensorLambda")(facts_output)
facts_tensors = Reshape((max_sent_num, 2*input_h_units))(facts_tensors)

#facts_mask = Masking(mask_value=0.0, name="FactTensorMask")(facts_tensors)

## Question Module

- **What it does:** The Question Module (similarly to the input module) returns an embedded representation for each question. However, unlike the Input Module, the only output of the Question Module is the last hidden state of the GRU (the embedded representation). This again, is exactly the implementation defined in the beforementioned paper, but it makes sense, considering that all questions are 1 sentence long.
- **How it does it:** Similarly to the input module (spotting a trend here) the input has to be padded and masked since not all questions are of equal length. This time a 'vanilla' unidirectional GRU is used as there is only 1 sentence/sequence, and so we wouldn't benefit from a bibirectional architecture like in the Input Module. Given that the outputs of the GRU are exactly what we want, no further processing is required.

In [11]:
question_input = Input(shape=(max_question_len, word_vector_size), name="QuestionInput")

question_mask = Masking(mask_value=0.0, name="QuestionMask")(question_input)

question_output = GRU(units=hidden_units, dropout=dropout,
                      kernel_regularizer=regularizers.l2(regularization_val),
                      recurrent_regularizer=regularizers.l2(regularization_val),
                      name="QuestionGRU")(question_mask)

## Semantic Memory Module

- **What it does:** The task of the Episodic Memory Module (in simplified terms) is to take the Context and Question embeddings, returned by the Input and Question modules respectively, and a return the final memory representation to the Answer Module. There are many Attention Mechanisms and various implementation details so I will cover the details of the Episodic Memory Module used in this project in the final report.
- **How it does it:** Again, in simplified terms, this Module can be thought of to contain 2 'parts'. An *'Attention Mechanism'* and a *'Memory Update Mechanism'*. Given the 'facts' returned by the Input Module (embedded representation of each sentence in the context) and the Question Module output (which is also an embedded representation, but this time of a single sentence, the question), the Episodic Memory Module chooses which parts of the inputs (facts) to focus on through the attention mechanism. It then produces a ”memory” vector representation taking into account the question as well as the previous memory. Each iteration provides the module with newly relevant information about the input. This acts as a form of transitive reasoning, by effectively allowing the module to re-pay attention to parts of the context that it found to be relevant to answering the question. 

In [12]:
epm_output = EpisodicMemoryModule(units=hidden_units, memory_steps=num_memory_passes,
                                  emb_dim=word_vector_size, batch_size=batch_size,
                                  dropout=dropout)([facts_tensors, question_output]) 

# Answer Module/Model Definition

- **What it does:** The final module of the Dynamic Memory Network is the Answer Module. The task of the Answer Module is to produce an answer given the output of the Semantic Memory Module. Given the nature of the DMN (being modular and thus generalisable to many different types of tasks) once again there are many implementations of the Answer Module, and once again I will spare the details for the report and only give a brief outline.
- **How it does it:** In this project we trained the network in a supervised setting. In the SQuAD dataset each answer is a span of the context meaning that the answer can be retrieved directly from the context as a continuous string. Thus the task of predicting the answer can be simplified to training the model to predict the start and end indices of the context, as this would allow us to directly extract the answer from the context. The objective/loss function is the sum of the categorical cross-entropy errors over the start/end index probability vectors (*start_idx_probs* and *end_idx_probs*.) 

In [13]:
# Defining the Answer Module
start_idx_probs = Dense(units=output_dim, activation="softmax", name="StartIdxProbs",
                        kernel_regularizer=regularizers.l2(regularization_val))(epm_output) 

end_idx_probs = Dense(units=output_dim, activation="softmax", name="EndIdxProbs",
                      kernel_regularizer=regularizers.l2(regularization_val))(epm_output)


# Defining the Model Architecture. 2 models, single-gpu model and multi-gpu model.
DMN_model = Model(inputs=[context_input, sent_end_idx_input, question_input],
                  outputs=[start_idx_probs, end_idx_probs])

DMN_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), 
                                                      metrics=['categorical_accuracy'])


#multi_gpu_DMN_model = multi_gpu_model(DMN_model, gpus=2)

##multi_gpu_DMN_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), 
                                                      #metrics=['categorical_accuracy'])

print(DMN_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ContextInput (InputLayer)       (None, 854, 50)      0                                            
__________________________________________________________________________________________________
ContextMask (Masking)           (None, 854, 50)      0           ContextInput[0][0]               
__________________________________________________________________________________________________
ContextBiGRU (Bidirectional)    (None, 854, 50)      11400       ContextMask[0][0]                
__________________________________________________________________________________________________
QuestionInput (InputLayer)      (None, 60, 50)       0                                            
__________________________________________________________________________________________________
FactTensor

# Training the Dynamic Memory Network


In [14]:
"""y_answer_start, y_answer_end = (get_answer_span(answer_start[0:batch_size], answer_end[0:batch_size]))
padded_contexts = get_padded_inputs(tokenized_contexts[0:batch_size], string_type="context")
padded_questions = get_padded_inputs(tokenized_questions[0:batch_size], string_type="question")
padded_sent_end_indices = pad_sequences(sent_end_indices[0:batch_size], max_sent_num,
                                                                    padding="post", dtype="int32")"""

#history = DMN_model.fit(x=[padded_contexts, padded_sent_end_indices, padded_questions],
 #                       y=[y_answer_start, y_answer_end],
  #                      batch_size=batch_size, validation_split=0.1)

#history_2 = multi_gpu_DMN_model.fit(x=[padded_contexts, padded_sent_end_indices, padded_questions],
                        #y=[y_answer_start, y_answer_end],
                        #batch_size=batch_size, validation_split=0.1)
    

'y_answer_start, y_answer_end = (get_answer_span(answer_start[0:batch_size], answer_end[0:batch_size]))\npadded_contexts = get_padded_inputs(tokenized_contexts[0:batch_size], string_type="context")\npadded_questions = get_padded_inputs(tokenized_questions[0:batch_size], string_type="question")\npadded_sent_end_indices = pad_sequences(sent_end_indices[0:batch_size], max_sent_num,\n                                                                    padding="post", dtype="int32")'

In [15]:
history_objects = []
batch_iteration = 0
for epoch in range(num_of_epochs):
    for batch_idx in range(num_of_batches):
        print("\nEpoch %d/%d     Batch %d/%d" % (epoch+1, num_of_epochs, batch_iteration+1, num_of_batches))
        start, end = (batch_idx*batch_size, (batch_idx+1)*batch_size)
    
        y_answer_start, y_answer_end = (get_answer_span(answer_start[start:end], answer_end[start:end]))
        padded_contexts = get_padded_inputs(tokenized_contexts[start:end], string_type="context")
        padded_questions = get_padded_inputs(tokenized_questions[start:end], string_type="question")
        padded_sent_end_indices = pad_sequences(sent_end_indices[start:end], max_sent_num,
                                                                    padding="post", dtype="int32")

        history = DMN_model.fit(x=[padded_contexts, padded_sent_end_indices, padded_questions],
                                y=[y_answer_start, y_answer_end],
                                batch_size=batch_size, validation_split=0.1)
        
        history_objects.append(history)
        batch_iteration+=1
        del start, end, y_answer_start, y_answer_end, padded_contexts, padded_questions, padded_sent_end_indices

    batch_iteration = 0 # Reset batch iteration counter for each new epoch"""


Epoch 1/1     Batch 1/678


C:\Users\Lukas Buteliauskas\Anaconda3_\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 14s 121ms/step - loss: 16.5179 - StartIdxProbs_loss: 8.2257 - EndIdxProbs_loss: 8.2238 - StartIdxProbs_categorical_accuracy: 0.0000e+00 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 16.5209 - val_StartIdxProbs_loss: 8.2304 - val_EndIdxProbs_loss: 8.2228 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 2/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 16ms/step - loss: 16.5007 - StartIdxProbs_loss: 8.2256 - EndIdxProbs_loss: 8.2073 - StartIdxProbs_categorical_accuracy: 0.0000e+00 - EndIdxProbs_categorical_accuracy: 0.0087 - val_loss: 16.4741 - val_StartIdxProbs_loss: 8.2027 - val_EndIdxProbs_loss: 8.2044 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 3/678
Train on 115 s


Epoch 1/1     Batch 18/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 15ms/step - loss: 15.8968 - StartIdxProbs_loss: 7.9066 - EndIdxProbs_loss: 7.9208 - StartIdxProbs_categorical_accuracy: 0.0000e+00 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 15.6829 - val_StartIdxProbs_loss: 7.7450 - val_EndIdxProbs_loss: 7.8676 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 19/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 15.8026 - StartIdxProbs_loss: 7.8740 - EndIdxProbs_loss: 7.8581 - StartIdxProbs_categorical_accuracy: 0.0000e+00 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 15.8018 - val_StartIdxProbs_loss: 7.8918 - val_EndIdxProbs_loss: 7.8384 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1


Epoch 1/1     Batch 35/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 15ms/step - loss: 14.3137 - StartIdxProbs_loss: 7.0557 - EndIdxProbs_loss: 7.1558 - StartIdxProbs_categorical_accuracy: 0.0609 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 14.5054 - val_StartIdxProbs_loss: 7.3054 - val_EndIdxProbs_loss: 7.0952 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 36/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 15ms/step - loss: 14.5077 - StartIdxProbs_loss: 7.1416 - EndIdxProbs_loss: 7.2612 - StartIdxProbs_categorical_accuracy: 0.0174 - EndIdxProbs_categorical_accuracy: 0.0087 - val_loss: 14.4081 - val_StartIdxProbs_loss: 7.2241 - val_EndIdxProbs_loss: 7.0763 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 3

...


Epoch 1/1     Batch 341/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 14.0354 - StartIdxProbs_loss: 6.7972 - EndIdxProbs_loss: 7.0727 - StartIdxProbs_categorical_accuracy: 0.0609 - EndIdxProbs_categorical_accuracy: 0.0261 - val_loss: 13.8482 - val_StartIdxProbs_loss: 6.9142 - val_EndIdxProbs_loss: 6.7686 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 342/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.8868 - StartIdxProbs_loss: 6.8113 - EndIdxProbs_loss: 6.9100 - StartIdxProbs_categorical_accuracy: 0.0174 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 13.1168 - val_StartIdxProbs_loss: 6.4277 - val_EndIdxProbs_loss: 6.5238 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0769

Epoch 1/1     Batch 343


Epoch 1/1     Batch 358/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.7051 - StartIdxProbs_loss: 6.7219 - EndIdxProbs_loss: 6.8194 - StartIdxProbs_categorical_accuracy: 0.0348 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 13.6433 - val_StartIdxProbs_loss: 6.2356 - val_EndIdxProbs_loss: 7.2439 - val_StartIdxProbs_categorical_accuracy: 0.2308 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 359/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.7096 - StartIdxProbs_loss: 6.7160 - EndIdxProbs_loss: 6.8299 - StartIdxProbs_categorical_accuracy: 0.0435 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 14.5033 - val_StartIdxProbs_loss: 7.2012 - val_EndIdxProbs_loss: 7.1385 - val_StartIdxProbs_categorical_accuracy: 0.0769 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 360


Epoch 1/1     Batch 375/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.4835 - StartIdxProbs_loss: 6.6490 - EndIdxProbs_loss: 6.6721 - StartIdxProbs_categorical_accuracy: 0.0348 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 13.8330 - val_StartIdxProbs_loss: 6.8059 - val_EndIdxProbs_loss: 6.8648 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 376/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.4059 - StartIdxProbs_loss: 6.5368 - EndIdxProbs_loss: 6.7068 - StartIdxProbs_categorical_accuracy: 0.0522 - EndIdxProbs_categorical_accuracy: 0.0000e+00 - val_loss: 14.3774 - val_StartIdxProbs_loss: 7.1050 - val_EndIdxProbs_loss: 7.1102 - val_StartIdxProbs_categorical_accuracy: 0.0769 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch


Epoch 1/1     Batch 392/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.6268 - StartIdxProbs_loss: 6.6960 - EndIdxProbs_loss: 6.7697 - StartIdxProbs_categorical_accuracy: 0.0348 - EndIdxProbs_categorical_accuracy: 0.0087 - val_loss: 13.7495 - val_StartIdxProbs_loss: 6.7234 - val_EndIdxProbs_loss: 6.8650 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 393/678
Train on 115 samples, validate on 13 samples
Epoch 1/1
115/115 [==============================] - 2s 14ms/step - loss: 13.3480 - StartIdxProbs_loss: 6.5564 - EndIdxProbs_loss: 6.6305 - StartIdxProbs_categorical_accuracy: 0.0261 - EndIdxProbs_categorical_accuracy: 0.0087 - val_loss: 14.4996 - val_StartIdxProbs_loss: 7.2048 - val_EndIdxProbs_loss: 7.1337 - val_StartIdxProbs_categorical_accuracy: 0.0000e+00 - val_EndIdxProbs_categorical_accuracy: 0.0000e+00

Epoch 1/1     Batch 394

ResourceExhaustedError: OOM when allocating tensor with shape[854,115,50] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: ContextBiGRU/transpose = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _class=["loc:@ContextBiGRU/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](ContextMask/mul, training/Adam/gradients/ContextBiGRU/transpose_2_grad/InvertPermutation)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


### Saving the Model 

In [ ]:
DMN_model.save('DMN_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model identical to the previous one
DMN_model_loaded = load_model('DMN_model.h5')

### Testing the Network

In [ ]:
"""Notes before testing:
    Have a careful look at the 'get_facts' method to get an idea of how training and testing
    are different. We must do 3 things:
    1. Assign value to test_send_end_indices
    2. Set 'training' to False
    3. Set 'batch_size' parameter in .predict() to be the number of smaples we are testing on. 
    4. Only do 1 test. Otherwise modify 'get_facts'"""

test_batch_size=20

test_contexts = None

context_inputs = None
indices_inputs = None
question_inputs = None

test_outputs = DMN_model.predict([context_inputs, indices_inputs, question_inputs], batch_size=test_batch_size)

In [ ]:
#Test of the context model
"""
context_model = Model(inputs=[context_input, sent_end_idx_input], outputs=facts_mask)

#Setting up test variables
padded_contexts = get_padded_inputs(tokenized_contexts[0:batch_size], string_type="context")
padded_sent_end_indices = pad_sequences(sent_end_indices[0:batch_size], max_sent_num, padding="post", dtype="int32")
y_test = tf.zeros((batch_size, max_sent_num, word_vector_size), dtype=tf.float32)

context_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate),
                      metrics=['accuracy', "categorical_accuracy"])
history = context_model.fit(x=[padded_contexts, padded_sent_end_indices], y=y_test,
                        batch_size=None, validation_split=0.1, steps_per_epoch=1, validation_steps=1)
predictions = context_model.predict([padded_contexts, padded_sent_end_indices], batch_size=batch_size)
"""